In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading the data 

In [2]:
df = pd.read_csv("/kaggle/input/marine/NOAAs_MDMAP_Sample_Data_2015-2018.csv")

Data Preprocessing 
* Dropping the null values 
* Filtering dataset with the amount of debris grouped by regions.



In [3]:
df.isnull().sum()
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [4]:
df.drop(["FID", "index"], axis = 1, inplace=True)

In [5]:
df.head()

In [6]:
pd.set_option("display.max_columns", None)

In [7]:
# Filtering dataset
lst = df.filter(like='Flux').columns
df_new = df.drop(lst, axis = 1)
df_new.drop(["Rope_Net_Pieces__non_nylon____F", "Disposable_Cigarette_Lighters__"], axis = 1, inplace=True)
# List with only debris 
df_num_col = df_new.columns[32: -2]
# List with Region details 
df_txt_col = df_new.columns[4:21]

In [8]:
df_num = df_new[df_num_col]
df_num.head()

# Visualising accumulation of debris diversity

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = '16'
labels = df_num.columns 
fig, ax1 = plt.subplots(figsize = (25,20)) 
ax1.pie(df_num.sum(), startangle=90) 
ax1.legend(labels = labels, loc = "upper right") 
plt.title("Distribution of accumulation of different types of Marine Debris", fontsize = 24)
plt.tight_layout() 
plt.show()

In [10]:
df_num_text = df_new[df_txt_col]
df_num_text.drop(["Survey_ID","MDMAP_ID__","Shoreline_Name", "Slope", "Width", "Length", "Time_of_Low_Tide"], axis = 1, inplace=True)

In [11]:
# Merging locations and types of debris
new = df_num_text.merge(df_num, left_index=True, right_index=True)
new.head()

In [12]:
new.info()

In [13]:
# Sorting the data with timestamps 
new.sort_values("Start_Time", inplace=True)
new = new.sort_values("Start_Time")

In [14]:
new.head()

In [15]:
new = new.reset_index()

In [16]:
new.drop("index", axis=1, inplace=True)

# Classifying and grouping the debris on basis of its composition

In [17]:
# Grouping litter that contains Plastic
new["Plastic"] = new["Plastic"] + new["Hard_Plastic_Fragments"] + new["Foamed_Plastic_Fragments"] + new["Filmed_Plastic_Fragments"] + new["Plastic_Beverage_Bottles"] + new["Other_Jugs_Containers"] + new["Bottle_Container_Caps"] + new["Cups"] + new["Straws"] + new["Balloons_Mylar"] + new["Personal_Care_Products"] + new["Plastic_Utensils"]

In [18]:
# Grouping litter that is caused through food and drinks 
new["Food_Wrappers"] = new["Food_Wrappers"] + new["Glass"] + new["Jars"] + new["Glass_Beverage_Bottles"] + new["Glass_Fragments"] + new["Glass_Other"]

In [19]:
# Grouping litter that is caused through Cigarettes 
new["Cigarettes"] = new["Cigar_Tips"] + new["F6_Pack_Rings"] + new["Disposable_Cigarette_Lighters"]

In [20]:
# Grouping litter caused by the Fishermen  
new["Plastic_Rope_Net"] = new["Buoys___Floats"]+ new["Fishing_Lures___Line"] + new["Plastic_Rope_Net"]

In [21]:
# Grouping litter which is contains metals and glasses 
new["Metal"] = new["Metal"] + new["Aluminum_Tin_Cans"] + new["Aerosol_Cans"] + new["Metal_Fragments"] + new["Metal_Other"]

In [22]:
# Grouping litter which is contains Rubber as a primary material
new["Rubber"] = new["Rubber_Fragments"] + new["Rubber_Other"] + new["Tires"] + new["Flip_Flops"] + new["Rubber_Gloves"] + new["Rubber"]

In [23]:
# Grouping debris which is caused due to construction sites
new["Lumber_Building_Material"] = new["Lumber_Building_Material"] + new["Processed_Lumber_Other"] 

In [24]:
# Grouping litter which is contains Fabric
new["Cloth_Fabric"] = new["Cloth_Fabric"] + new["Clothing___Shoes"] + new["Towels_Rags"] + new["Rope_Net_Pieces__non_nylon_"] + new["Fabric_Pieces"] + new["Cloth_Fabric_Other"]

In [25]:
new.columns

In [26]:
new.drop(['Hard_Plastic_Fragments',
       'Foamed_Plastic_Fragments', 'Filmed_Plastic_Fragments','Plastic_Beverage_Bottles', 'Other_Jugs_Containers',
       'Bottle_Container_Caps', 'Cigar_Tips', 'Disposable_Cigarette_Lighters', 'F6_Pack_Rings', 'Bags', 'Buoys___Floats', 'Fishing_Lures___Line', 'Cups',
       'Plastic_Utensils', 'Straws', 'Balloons_Mylar',
       'Personal_Care_Products', 'Plastic_Other', 
       'Personal_Care_Products', 'Plastic_Other', 'Aluminum_Tin_Cans',
       'Aerosol_Cans', 'Metal_Fragments', 'Metal_Other', 'Glass',
       'Glass_Beverage_Bottles', 'Jars', 'Glass_Fragments', 'Glass_Other','Flip_Flops', 'Rubber_Gloves', 'Tires', 'Balloons_Latex',
       'Rubber_Fragments', 'Rubber_Other', 'Processed_Lumber',  'Clothing___Shoes', 'Gloves__non_rubber_', 'Towels_Rags',
       'Rope_Net_Pieces__non_nylon_', 'Fabric_Pieces', 'Cloth_Fabric_Other'], axis =1 , inplace = True)

In [27]:
new.head()

In [28]:
import plotly.express as px

# Visualising the accumulation of debris over time using choropleth map.
* Kindly click on the play button to see the accumulation over time. 
* The size of the plot is directly proportional to the denisty of accumulation in the respective region.
* The colors label on right indicate the range of average quantity of debris in the marked region.

# Plastic Debris

In [29]:
import plotly.express as px
import geopandas as gpd
import plotly.express as px
import geopandas as gpd

fig = px.scatter_geo(new, lat=new["Latitude_Start"],lon=new["Longitude_Start"],hover_name='Plastic', animation_frame = "Start_Time",size="Plastic", title = "Accumulation of plastic debris over time", width=1000, height = 1000)
fig_px = px.choropleth(new,     
                       
                       locations = "Country",
                       color="Plastic",
                       locationmode='country names', 
                       animation_frame="Start_Time",
                       #animation_group="Country",
                       hover_data={'Plastic':False, 
                                  })

fig.add_trace(
    fig_px.data[0]
)
fig.show()


# Food and drinks debris 

In [30]:
import plotly.express as px
import geopandas as gpd

fig = px.scatter_geo(new, lat=new["Latitude_Start"],lon=new["Longitude_Start"],hover_name='Food_Wrappers', animation_frame = "Start_Time",size="Food_Wrappers", title = "Accumulation of food debris over time", width=1000, height = 1000)
fig_px = px.choropleth(new,     
                       locations = "Country",
                       color="Food_Wrappers",
                       locationmode='country names', 
                       animation_frame="Start_Time",
                       #animation_group="Country",
                       hover_data={'Food_Wrappers':False, 
                                  })

fig.add_trace(
    fig_px.data[0]
)
fig.show()

# Metal debris

In [31]:
fig = px.scatter_geo(new, lat=new["Latitude_Start"],lon=new["Longitude_Start"],hover_name='Metal', animation_frame = "Start_Time",size="Metal", title = "Accumulation of metal and glass debris over time", width=1000, height = 1000)
fig_px = px.choropleth(new,     
                       locations = "Country",
                       color="Metal",
                       locationmode='country names', 
                       animation_frame="Start_Time",
                       #animation_group="Country",
                       hover_data={'Metal':False, 
                                  })

fig.add_trace(
    fig_px.data[0]
)
fig.show()

In [32]:
fig = px.scatter_geo(new, lat=new["Latitude_Start"],lon=new["Longitude_Start"],hover_name="Plastic_Rope_Net", animation_frame = "Start_Time",size="Plastic_Rope_Net", title = "Accumulation of debris caused by fishermen over time", width=1000, height = 1000)
fig_px = px.choropleth(new,     
                       locations = "Country",
                       color="Plastic_Rope_Net",
                       locationmode= 'country names', 
                       animation_frame="Start_Time",
                       #animation_group="Country",
                       hover_data={"Plastic_Rope_Net":False, 
                                  })

fig.add_trace(
    fig_px.data[0]
)
fig.show()